In [3]:
from langchain.llms import OpenAI
llm = OpenAI(temperature = 0.5) # temperature is a measure of randomness
print(llm.predict('How old was donald trump in 1987?'))




Donald Trump was 41 years old in 1987.


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (AIMessage,HumanMessage,SystemMessage)

chat = ChatOpenAI(temperature = 0.5)
print(chat.predict_messages([HumanMessage(content="whats 9+10 (hint, the answer I want is referencing a meme)")]))

content='21 (the meme is "What\'s 9+10? 21, you stupid.")' additional_kwargs={} example=False


In [5]:
chat.predict("whats 9+10 (hint, the answer I want is referencing a meme)")

'21 (the answer from the "What\'s 9+10?" meme)'